In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

# NOTE: librosa dependencies apparently require specific versions of numpy, try numpy==1.21.4
import librosa
import librosa.display

import IPython.display
import scipy

Dataset explored is UrbanSound8K:

J. Salamon, C. Jacoby and J. P. Bello, "A Dataset and Taxonomy for Urban Sound Research", 
22nd ACM International Conference on Multimedia, Orlando USA, Nov. 2014.

https://urbansounddataset.weebly.com/urbansound8k.html

https://zenodo.org/record/1203745

In [ ]:
# Data should be placed in the "large_data/" directory, which is not staged in the git repo
metadata = pd.read_csv('./large_data/UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.info()
display(metadata)

In [ ]:
# Function to load wav file with librosa, given a row from metadata table
def load_data(meta_row):
    filename = meta_row['slice_file_name']
    filepath = f'large_data/UrbanSound8K/audio/fold{meta_row["fold"]}/'
    return librosa.load(filepath+filename,sr=None)

The various audiofiles do not exhibit a uniform length or sampling rate. As such we aim to do some pre-classification analysis to generate a uniform observable vector.

For this reason we are going to take inspiration from the idea of a music "equalizer" scale. We will break down the fourier transform of each audio section into a number of human audible frequencies, as well as the sub-audible and supra-audible sections. 

In [ ]:
row_ind = metadata.loc[514]
test_y, sr = load_data(row_ind)

# row_ind = 100
# # row_ind = metadata_gun_shot.iloc[row_ind]
# test_y, sr = load_data(metadata_gun_shot.iloc[row_ind])

load_data(metadata.loc[1])

len(test_y)

t = round(row["end"]-row["start"],3)
dt = t/len(test_y)
print(dt)

k = np.fft.fftfreq(len(test_y), d=dt)
print(k[1])

yk = np.fft.fft(test_y)
mag_yk = np.abs(yk)
mag_yk = mag_yk[:len(mag_yk)//2]

test_y2 = np.sqrt(test_y*test_y)
yk2 = np.fft.fft(test_y2)
mag_yk2 = np.abs(yk2)
mag_yk2 = mag_yk2[:len(mag_yk2)//2]

filttest = yk * np.exp(-k*k/10000000)
test_back = np.fft.ifft(filttest)

test_y.max()

np.sqrt(test_y2.mean())

Crestfactor = test_y.max()/np.sqrt(test_y2.mean())
Crestfactor

After initial exploration of the data I noticed that a number of the audio samples have significant "room noise" present. In an attempt to clean this up I'm taking the hilbert transform (extracts the instantaneous amplitude of a signal). Smoothing this transform and dividing by the root mean square power should then amplify the parts of the signal that are large in amplitude, while minimizing the areas that are simply a constant amplitude "hum". Low crest factor systems will be largely unaffected as the root mean square will be similar to the root mean square of the signal for such signals. Finally we ensure that the maximum amplitude of the signal is scaled to be equal to the input signal.

In [1120]:
testwav = test_y * np.abs(np.abs(scipy.signal.hilbert(test_y)))/np.sqrt((test_y*test_y).mean())
testwav *= test_y.max() / testwav.max()

In [ ]:
hil = np.abs(scipy.signal.hilbert(test_y))
plt.plot(hil)
# hil = np.abs(scipy.signal.hilbert(testwav2))
# plt.plot(hil)

In [ ]:
hilk = np.fft.fft(hil)
hilk *= np.exp(-k*k / (2*10**2))
hil2 = np.fft.ifft(hilk)
hil2 = np.abs(hil2)
hil2 *= hil.max() / hil2.max()
plt.plot(hil)
plt.plot(np.abs(hil2))

In [ ]:
testwav2 = test_y * hil2/np.sqrt((test_y*test_y).mean())
testwav2 *= test_y.max() / testwav2.max()

Check the audio samples against one another to see if our attempts at minimizing room noise were successful.

In [ ]:
IPython.display.Audio(test_y,rate=sr)

In [ ]:
IPython.display.Audio(testwav / testwav.max() * test_y.max(),rate=sr)

In [ ]:
IPython.display.Audio(testwav2 / testwav2.max() * test_y.max(),rate=sr)

Note that the version with the filtered hilbert transform acting as the instantaneous amplitude has less distortion.

We now generate the equilizer values by manually summing over the relevent portions of fourier space.

In [ ]:
eq_cutoffs = [20.0, 40.0, 80.0, 160.0, 300.0, 600.0, 1200.0, 2400.0, 5000.0, 10000.0, 20000.0, np.inf]
equilizer = np.zeros(12)
equilizer2 = np.zeros(12)
equilizer3 = np.zeros(12)
# equilizer4 = np.zeros(12)

ykclean = np.fft.fft(testwav)
mag_ykclean = np.abs(ykclean)
mag_ykclean = mag_ykclean[:len(mag_ykclean)//2]

ykclean2 = np.fft.fft(testwav2)
mag_ykclean2 = np.abs(ykclean2)
mag_ykclean2 = mag_ykclean2[:len(mag_ykclean2)//2]

# ykclean3 = np.fft.fft(testwav3)
# mag_ykclean3 = np.abs(ykclean3)
# mag_ykclean3 = mag_ykclean3[:len(mag_ykclean3)//2]

for i in range(0,12):
    index = 0
    num = 0
    while ((index < len(mag_yk)) & (k[index] < eq_cutoffs[i])):
        equilizer[i] += mag_yk[index]
        equilizer2[i] += mag_ykclean[index]
        equilizer3[i] += mag_ykclean2[index]
#         equilizer4[i] += mag_ykclean3[index]
        index += 1
        num += 1
    equilizer[i] /= num
    equilizer2[i] /= num
    equilizer3[i] /= num
#     equilizer4[i] /= num
    
equilizer /= equilizer.max()
equilizer2 /= equilizer2.max()
equilizer3 /= equilizer3.max()
# equilizer4 /= equilizer4.max()

In [ ]:
print(equilizer)
print(equilizer2)
print(equilizer3)

Sanity check with some plots. We notice that the unfiltered version of our clean up step displays distortion in the numerical data as well as testing by ear.

In [ ]:
plt.scatter(range(0,12),equilizer)
plt.scatter(range(0,12),equilizer2)
plt.scatter(range(0,12),equilizer3)

In [ ]:
plt.scatter(range(0,12),np.log(equilizer)/np.log(10.0))
plt.scatter(range(0,12),np.log(equilizer2)/np.log(10.0))
plt.scatter(range(0,12),np.log(equilizer3)/np.log(10.0))

Make a function to do the cleanup and save the results as a nice, compact csv file.

In [ ]:
def make_filt_eq_csv(filenm):
    file = open(filenm,'w+')
    file.write("class,eq_0,eq_20,eq_40,eq_80,eq_160,eq_300,eq_600,eq_1200,eq_2400,eq_5000,eq_10000,eq_20000,crestfactor,fold\n")
    
    eq_cutoffs = [20.0, 40.0, 80.0, 160.0, 300.0, 600.0, 1200.0, 2400.0, 5000.0, 10000.0, 20000.0, np.inf]
    
    for row_ind in range(0,len(metadata)):
        y,sr = load_data(metadata.loc[row_ind])
        classifier = metadata['class'][row_ind]
        foldinfo = metadata['fold'][row_ind]
        t = round(row["end"]-row["start"],3)
        dt = t/len(y)
        k = np.fft.fftfreq(len(y), d=dt)
#         k = k[:len(k)//2]
        
        
        hil = np.abs(scipy.signal.hilbert(y))
        hilk = np.fft.fft(hil)
        hilk *= np.exp(-k*k / (2*10**2))
        hil2 = np.fft.ifft(hilk)
        hil2 = np.abs(hil2)
        hil2 *= hil.max() / hil2.max()
        filt_y = y * hil2/np.sqrt((y*y).mean())
        filt_y *= y.max() / filt_y.max()
        y_sq = filt_y*filt_y
        Cr = filt_y.max() / np.sqrt(y_sq.mean())
        
        yk = np.fft.fft(filt_y)
        mag_yk = np.abs(yk)
        mag_yk = mag_yk[:len(mag_yk)//2]
        
        equilizer = np.zeros(12)
        for i in range(0,12):
            index = 0
            num = 0
            while ((index < len(mag_yk)) & (k[index] < eq_cutoffs[i])):
                equilizer[i] += mag_yk[index]
                index += 1
                num += 1
            equilizer[i] /= num
            
        file.write(classifier)
        for i in range(0,12):
            file.write(","+str(equilizer[i]))
        
        file.write(","+str(Cr))
        file.write(","+str(foldinfo)+"\n")
    file.close()

In [ ]:
make_filt_eq_csv('./large_data/filt_equilizer_data.csv')

Check that everything worked and we can load our file as expected.

In [ ]:
eq_df = pd.read_csv('./large_data/filt_equilizer_data.csv')
display(eq_df)